## Tool by tool walkthrough

A step by step runthrough of the separate veg2hab tools

In [1]:
from veg2hab.main import run
from veg2hab.io.common import (
    AccessDBInputs,
    StackVegKarteringInputs,
    ApplyDefTabelInputs,
    ApplyMozaiekInputs,
    ApplyFunctioneleSamenhangInputs,
)
from veg2hab.io.cli import CLIInterface

import geopandas as gpd

CLIInterface.get_instance()

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
step_1 = AccessDBInputs(
    shapefile="../testing/Rottige_Meenthe_Brandemeer_2013/vlakken.shp",
    elmid_col="ElmID",
    access_mdb_path="../testing/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb",
    output="../data/notebook_data/tool_by_tool/1.gpkg"
)

step_3 = ApplyDefTabelInputs(
    shapefile=str(step_1.output),
    output="../data/notebook_data/tool_by_tool/3.gpkg"
)

step_4 = ApplyMozaiekInputs(
    shapefile=str(step_3.output),
    output="../data/notebook_data/tool_by_tool/4.gpkg"
)

step_5 = ApplyFunctioneleSamenhangInputs(
    shapefile=str(step_4.output),
    output="../data/notebook_data/tool_by_tool/5.gpkg"
)

## No changes run

In [3]:
print("Starting step 1")
run(step_1)
print("Starting step 3")
run(step_3)
print("Starting step 4")
run(step_4)
print("Starting step 5")
run(step_5)
gpd.read_file(step_5.output).head()

## With changes run

In [4]:
print("----- Starting step 1 -----")
run(step_1)
gdf = gpd.read_file(step_1.output)
# Verkleinen voor speeeeeed
gdf = gdf.head(10)
gdf.loc[0, "EDIT_SBB1"] = "1a1a"
gdf.loc[0, "EDIT_VvN1"] = "1aa1a"
gdf.to_file(step_1.output, driver="GPKG", layer="main")
print("----- Starting step 3 -----")
run(step_3)
gdf = gpd.read_file(step_3.output)
gdf.loc[0, "EDIT_Habtype1"] = "H1234"
gdf.loc[0, "EDIT_Kwal1"] = "M"
gdf.loc[0, "EDIT_Opm1"] = "test1"
gdf.to_file(step_3.output, driver="GPKG", layer="main")
print("----- Starting step 4 -----")
run(step_4)
gdf = gpd.read_file(step_4.output)
gdf.loc[1, "EDIT_Habtype1"] = "H4321"
gdf.loc[1, "EDIT_Kwal1"] = "G"
gdf.loc[1, "EDIT_Opm1"] = "test2"
gdf.to_file(step_4.output, driver="GPKG", layer="main")
print("----- Starting step 5 -----")
run(step_5)
done = gpd.read_file(step_5.output)
print(done.iloc[0].SBB1)
print(done.iloc[0].Habtype1)
print(done.iloc[1].Habtype1)
done.head()

----- Starting step 1 -----
mdbtools v1.0.0


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


----- Starting step 3 -----


----- Starting step 4 -----


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


----- Starting step 5 -----


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


1a1a
H1234
H4321


,Area,Opm,Datum,ElmID,_Samnvttng,_LokVegTyp,_LokVrtNar,Habtype1,Perc1,Opp1,...,SBB4,VvN4,_Status4,_Uitleg4,_SBBdftbl4,_VvNdftbl4,_Mits_opm4,_Mozk_opm4,_MozkPerc4,geometry
0,3761.736281,None,None,28049,"90.0% H1234, 10.0% H7140_B","09a2-6 (90%), 09a2-1 (10%)",Lokale typologie is primair vertaald naar SBB,H1234,90,3385.562652979308,...,None,None,None,None,None,None,None,None,None,"POLYGON ((189433.818 538314.302, 189439.215 53..."
1,10442.658431,None,None,15034,100.0% H4321,50a-1 (100%),Lokale typologie is primair vertaald naar SBB,H4321,100.0,10442.658431475804,...,None,None,None,None,None,None,None,None,None,"POLYGON ((190568.500 539171.122, 190538.023 53..."
2,2174.098680,None,None,13503,100.0% H7140_B,"09a2-1 (90%), 09a2-6 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,1956.68881156644,...,None,None,None,None,None,None,None,None,None,"POLYGON ((189538.104 539409.332, 189532.399 53..."
3,18878.050297,None,None,15075,"90.0% HXXXX, 10.0% H7140_B","09b-4 (80%), 09-13 (10%), 09a2-2 (10%)",Lokale typologie is primair vertaald naar SBB,HXXXX,80,15102.44023721278,...,None,None,None,None,None,None,None,None,None,"POLYGON ((190801.313 539561.740, 190819.617 53..."
4,4950.611630,None,None,20502,"61.0% H0000, 39.0% HXXXX","08b3-3 (58%), 08c-6 (29%), 09-11 (10%), 05d5-1...",Lokale typologie is primair vertaald naar SBB,HXXXX,29,1435.67737260958,...,5d5a,5bc5,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn op geen en...,None,None,,,,"POLYGON ((190497.231 537724.945, 190505.062 53..."


## Doing illegal things

In [16]:
print("----- Starting step 1 -----")
run(step_1)
gdf = gpd.read_file(step_1.output)
# Verkleinen voor speeeeeed
gdf = gdf.head(10)
gdf.loc[0, "EDIT_SBB3"] = "1a1a"
gdf.loc[0, "EDIT_VvN3"] = "1aa1a"
gdf.loc[0, "EDIT_perc3"] = "10"

gdf.loc[1, "EDIT_SBB2"] = "1a1a"
gdf.loc[1, "EDIT_VvN2"] = "1aa1a"
gdf.loc[1, "EDIT_perc2"] = "10"
gdf.to_file(step_1.output, driver="GPKG", layer="main")
print("----- Starting step 3 -----")
run(step_3)
gdf = gpd.read_file(step_3.output)
gdf.loc[0, "EDIT_Habtype3"] = "H1234"
gdf.loc[0, "EDIT_Kwal3"] = "M"
gdf.loc[0, "EDIT_Opm3"] = "test1"
gdf.to_file(step_3.output, driver="GPKG", layer="main")
print("----- Starting step 4 -----")
run(step_4)
gdf = gpd.read_file(step_4.output)
gdf.loc[1, "EDIT_Habtype2"] = "H4321"
gdf.loc[1, "EDIT_Kwal2"] = "G"
gdf.loc[1, "EDIT_Opm2"] = "test2"
gdf.to_file(step_4.output, driver="GPKG", layer="main")
print("----- Starting step 5 -----")
run(step_5)
done = gpd.read_file(step_5.output)
print(done.iloc[0].SBB3)
print(done.iloc[0].Habtype3)
print(done.iloc[1].Habtype2)
done.head()

----- Starting step 1 -----


mdbtools v1.0.0


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


----- Starting step 3 -----


----- Starting step 4 -----


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


----- Starting step 5 -----


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


1a1a
H1234
H4321


,Area,Opm,Datum,ElmID,_Samnvttng,_LokVegTyp,_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,Opm1,SBB1,VvN1,_Status1,_Uitleg1,_SBBdftbl1,_VvNdftbl1,_Mits_opm1,_Mozk_opm1,_MozkPerc1,Habtype2,Perc2,Opp2,Kwal2,Opm2,SBB2,VvN2,_Status2,_Uitleg2,_SBBdftbl2,_VvNdftbl2,_Mits_opm2,_Mozk_opm2,_MozkPerc2,Habtype3,Perc3,Opp3,Kwal3,Opm3,SBB3,VvN3,_Status3,_Uitleg3,_SBBdftbl3,_VvNdftbl3,_Mits_opm3,_Mozk_opm3,_MozkPerc3,Habtype4,Perc4,Opp4,Kwal4,Opm4,SBB4,VvN4,_Status4,_Uitleg4,_SBBdftbl4,_VvNdftbl4,_Mits_opm4,_Mozk_opm4,_MozkPerc4,geometry
0,3761.736281,None,None,28049,"100.0% H7140_B, 10.0% H1234","09a2-6 (90%), 09a2-1 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,3385.562652979308,G,,9a2a,9aa2a,KeuzeStatus.HABITATTYPE_TOEGEKEND,veg2hab heeft één habitattype gevonden waaraan dit vlak voldoet.,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar), TRUE",,H7140_B,10,376.173628108812,G,,9a2a,9aa2a,KeuzeStatus.HABITATTYPE_TOEGEKEND,veg2hab heeft één habitattype gevonden waaraan dit vlak voldoet.,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar), TRUE",,H1234,10.0,376.173628,M,test1\n\n,1a1a,1aa1a,KeuzeStatus.HANDMATIG_TOEGEKEND,Het habitattype is handmatig aangepast in een van de tussentijdse resultaten.,None,None,,,,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((189433.818 538314.302, 189439.215 538312.873, 189442.708 538313.508, 189447.946 538306.047, 189451.121 538303.189, 189468.108 538285.568, 189482.871 538266.359, 189492.396 538253.183, 18..."
1,10442.658431,None,None,15034,"100.0% HXXXX, 10.0% H4321",50a-1 (100%),Lokale typologie is primair vertaald naar SBB,HXXXX,100.0,10442.658431475804,X,Dit is mogelijk toch Hoogveenlandschap ondanks LBK code LvMP.\nDit is mogelijk toch Hoogveenlandschap ondanks LBK code LvMP.,50a,,KeuzeStatus.NIET_GEAUTOMATISEERD_CRITERIUM,Er zijn niet-geautomatiseerde mitsen/mozaiekregels gevonden; deze kunnen niet door veg2hab worden gecontroleerd. De gebruiker moet hier een handmatige controle uitvoeren.,"['50a', 213, 'H2190_A']\n['50a', 292, 'H3110']\n['50a', 311, 'H3130']\n['50a', 317, 'H3140']\n['50a', 324, 'H3150']\n['50a', 349, 'H3160']\n['50a', 360, 'H3260_A']\n['50a', 361, 'H3260_A']\n['50a'...",---\n---\n---\n---\n---\n---\n---\n---\n---\n---\n---\n---\n---\n---,"[50a, H2190_A, Geen mits (altijd waar), TRUE]\n[50a, H3110, Geen mits (altijd waar), TRUE]\n[50a, H3130, Geen mits (altijd waar), TRUE]\n[50a, H3140, Geen mits (altijd waar), TRUE]\n[50a, H3150, G...","[50a, H2190_A, zelfstndg/mozk G/M H2190_A., POSTPONE]\n[50a, H3110, zelfstndg/mozk G/M H3110., POSTPONE]\n[50a, H3130, zelfstndg/mozk G/M H3130., POSTPONE]\n[50a, H3140, zelfstndg G/M H3140., ...",,H4321,10,1044.26584314758,G,test2\n,1a1a,1aa1a,KeuzeStatus.HANDMATIG_TOEGEKEND,Het habitattype is handmatig aangepast in een van de tussentijdse resultaten.,None,None,,,,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((190568.500 539171.122, 190538.023 539149.444, 190537.840 539149.729, 190388.311 539381.844, 190392.667 539384.748, 190420.409 539402.887, 190433.900 539381.774, 190437.603 539375.979, 19..."
2,2174.098680,None,None,13503,100.0% H7140_B,"09a2-1 (90%), 09a2-6 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,1956.68881156644,G,,9a2a,9aa2a,KeuzeStatus.HABITATTYPE_TOEGEKEND,veg2hab heeft één habitattype gevonden waaraan dit vlak voldoet.,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar), TRUE",,H7140_B,10,217.4098679518266,G,,9a2a,9aa2a,KeuzeStatus.HABITATTYPE_TOEGEKEND,veg2hab heeft één habitattype gevonden waaraan dit vlak voldoet.,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar), TRUE",,None,NaN,NaN,None,None,None,None,None,None,Non

In [15]:
done.iloc[2]

Area                                                                                                                                                                                                           2174.1
Opm                                                                                                                                                                                                              None
Datum                                                                                                                                                                                                            None
ElmID                                                                                                                                                                                                           13503
_Samnvttng                                                                                                                                      